In [1]:
import requests
import json
from tqdm import tqdm_notebook as tqdm
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

headers = {
    'authority': 'be.wizzair.com',
    'accept': 'application/json, text/plain, */*',
    'origin': 'https://wizzair.com',
    'x-requestverificationtoken': 'ac8befaec3404c6f888b4e7a109b0dde',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'content-type': 'application/json;charset=UTF-8',
    'sec-fetch-site': 'same-site',
    'sec-fetch-mode': 'cors',
    'referer': 'https://wizzair.com/',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-GB,en;q=0.9,hu-HU;q=0.8,hu;q=0.7,en-US;q=0.6',
    'cookie': 'bm_sz=92C831F5A010DB837DD22BC7D9DABE14~YAAQXLP3SPLOrqpuAQAANtdOvgXgSn1vUEpWGqs2UclxWimcnPAda+G0xCsjy9W6SE35RuJnBKeN0MRUezmUBa2H4gPRZzjzLTz+XD7XeQVR2o7tYs0d5b8XTRzCntoct3fcOXJlFP2s1ncWdaTkbUduwTijFEi8GlPGhG3D9f8/BBaY13WwYCpZq2UoSc+kcg==; ak_bmsc=91169AFD7AE4CC6730EA6FD6A7D2033348F7B35CE12C000014E5E25D6E711664~plcwWxfqBvvl+k3QcFUXlS6t93dKz/HgsYwoCZVt/dOJUOV+hQXqZpqlCmTZALcre/M3v0lkjiWHx6cw9aaflo6lXwnZDnDMhR+Xr9y88aFWNt3FMgQd7Gry3F8qzxmpa/ZOP1S6YlJOhQ9R9MK1oir3Apd3cgRlpHo6tzRpt/3EA1J0/mauvgkS9hfCEsC1kvI5TcHkDkZqo5nK7N5AK+YyJ/yvFfw6Gft4O9h0EYJ6EdgyW9POwZAYm+BFPL+Ws8yXF8mEPSuFarcYtLfyJI94H8SiGrf6wai2yfYVZCbLy/NcBPipFK5qSE4oiWB/iMsgfT5XFq5iBB3HbxQux5Hg==; _hjid=03345ec6-a6df-4a81-b074-499401290e0d; ASP.NET_SessionId=nii5vjpxmf0inpfqzjtleyub; RequestVerificationToken=ac8befaec3404c6f888b4e7a109b0dde; _gcl_au=1.1.2040786257.1575150888; _fbp=fb.1.1575150888158.621637889; _ga=GA1.2.784960775.1575150888; _gid=GA1.2.201837638.1575150888; _gat_gtag_UA_2629375_25=1; _abck=AA1A827631DBC5B8238586DBFD4434C4~0~YAAQV7P3SJrYK61uAQAADaNivgJO9rc7kHrRGgD+sgw3HRNgSZ53US+StMf6mamlox4bhiT72P6I2KOYqY6ihSrTPlnAxCI1B/eNLne66uRyQYAm3H9KbHctljIxA/NzJCQvxHRz2nYgE7ARzmknspsOiMiPzedYoX1T0k2ePfFxfvqvXJ43c7hSibkeY1LvK0uVplnw7ikbzzp6QMs4NkkIHk8K/S+sgBnVCfVzZnEi/Wm/aFg3bDIHBrZZObyMWEGIEbqXNWAMs1Afq6KtRTbG3cjJj6yDVhiE7hE95B0wUFb3Iw92JL+Gf3iMpYwUjMUnhaB/7E4=~-1~-1~-1; bm_sv=4FDC6A6E2592514D9E5DA2834B24A75B~q4P0s24xVp0oiMg8lW12ECJxvcv4eMWh07nqKPaR3q3RBXSr4CayMJi6aU6PZZbEpSXHq8P8RZ6sJpCgmNFE98pJ+lCV71tHRo+w/+oTyxCtN0J7i1DDtZSoun+zAPJRZflUFXKpDWZxLiUDU5Rd2AmwRcFonFGx4ZDXdWGuwko=',
}

In [2]:
data = {"isFlightChange":False,
        "isSeniorOrStudent":False,
        "flightList":[{"departureStation":"BUD",
                       "arrivalStation":"CRL",
                       "departureDate":"2020-06-16"}],
        "adultCount":1,
        "childCount":0,
        "infantCount":0,
        "wdc":True}

In [3]:
base = datetime.today() + timedelta(days = 2)
date_list = [base + timedelta(days=x) for x in range(365)]

In [5]:
prices_dict = {}
arrival_stations = ["CRL", "EIN"]
df_list = []
for date in tqdm(date_list):
    data["flightList"][0]["departureDate"] = date.strftime("%Y-%m-%d")
    for arrival_station in arrival_stations:
        data["flightList"][0]["arrivalStation"] = arrival_station
        response = requests.post('https://be.wizzair.com/10.1.0/Api/search/search', headers=headers, data=json.dumps(data))
        try:
            resp_data = response.json()
            prices_dict["normal_price"]  = [resp_data["outboundFlights"][0]["fares"][3]["basePrice"]["amount"]]
            prices_dict["wdc_price"] = [resp_data["outboundFlights"][0]["fares"][0]["basePrice"]["amount"]]
        except:
            prices_dict["normal_price"] = np.nan
            prices_dict["wdc_price"] = np.nan
        prices_dict["Date"] = [date.strftime("%Y-%m-%d")]
        prices_dict["Departure"] = ["BUD"]
        prices_dict["Destination"] = [arrival_station]
        df_list.append(pd.DataFrame.from_dict(prices_dict))


In [6]:
final = pd.concat(df_list).reset_index(drop = True)

In [7]:
final

,normal_price,wdc_price,Date,Departure,Destination
0,19490.0,16190.0,2019-12-03,BUD,CRL
1,16190.0,12990.0,2019-12-03,BUD,EIN
2,5690.0,3190.0,2019-12-04,BUD,CRL
3,7290.0,3990.0,2019-12-04,BUD,EIN
4,25990.0,22690.0,2019-12-05,BUD,CRL
...,...,...,...,...,...
725,NaN,NaN,2020-11-29,BUD,EIN
726,NaN,NaN,2020-11-30,BUD,CRL
727,NaN,NaN,2020-11-30,BUD,EIN
728,NaN,NaN,2020-12-01,BUD,CRL
